For this challenge, you will need to choose a corpus of data from nltk or another source that includes categories you can predict and create an analysis pipeline that includes the following steps:

Data cleaning / processing / language parsing
Create features using two different NLP methods: For example, BoW vs tf-idf.
Use the features to fit supervised learning models for each feature set to predict the category outcomes.
Assess your models using cross-validation and determine whether one model performed better.
Pick one of the models and try to increase accuracy by at least 5 percentage points.

In [61]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn import neighbors
from nltk.corpus import gutenberg, stopwords
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import ensemble
import time

In [3]:
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


# Poetry
Explore poetry by William Blake and Walt Whitman, train model to recognize the author.

## Bag of Words

### Data processing

In [4]:
poems = gutenberg.raw('blake-poems.txt')
leaves = gutenberg.raw('whitman-leaves.txt')

In [5]:
print('\nRaw:\n', poems[0:200])


Raw:
 [Poems by William Blake 1789]

 
SONGS OF INNOCENCE AND OF EXPERIENCE
and THE BOOK of THEL


 SONGS OF INNOCENCE
 
 
 INTRODUCTION
 
 Piping down the valleys wild,
   Piping songs of pleasant glee,
 O


In [6]:
print('\nRaw:\n', leaves[0:200])


Raw:
 [Leaves of Grass by Walt Whitman 1855]


Come, said my soul,
Such verses for my Body let us write, (for we are one,)
That should I after return,
Or, long, long hence, in other spheres,
There to some g


In [7]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text

In [8]:
poems = text_cleaner(poems)
leaves = text_cleaner(leaves)

In [10]:
nlp = spacy.load('en')
poems_doc = nlp(poems)

In [11]:
leaves_doc = nlp(leaves)

In [12]:
poems_sents = [[sent, "Blake"] for sent in poems_doc.sents]
leaves_sents = [[sent, "Whitman"] for sent in leaves_doc.sents]

print(len(poems_sents))
print(len(leaves_sents))

498
5709


In [13]:
leaves_sents = leaves_sents[0:len(poems_sents)]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(poems_sents + leaves_sents)
sentences.head()

,0,1
0,"(SONGS, OF, INNOCENCE, AND, OF, EXPERIENCE, an...",Blake
1,"(INTRODUCTION, Piping, down, the, valleys, wil...",Blake
2,"(So, I, piped, with, merry, cheer, ., "")",Blake
3,"(Piper, ,, pipe, that, song, again)",Blake
4,"(;, "", So, I, piped, :, he, wept, to, hear, ., "")",Blake


In [17]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
poemswords = bag_of_words(poems_doc)
leaveswords = bag_of_words(leaves_doc)

# Combine bags to create a set of unique words.
common_words = set(poemswords + leaveswords)

In [18]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400
Processing row 450
Processing row 500
Processing row 550
Processing row 600
Processing row 650
Processing row 700
Processing row 750
Processing row 800
Processing row 850
Processing row 900
Processing row 950


,welcome,sunrise,affectionate,utter,sailing,receiv'd,finger,count,material,translate,...,lyca,brook,dazzling,risen,plant,boundless,anguish,ore,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(SONGS, OF, INNOCENCE, AND, OF, EXPERIENCE, an...",Blake
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(INTRODUCTION, Piping, down, the, valleys, wil...",Blake
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, I, piped, with, merry, cheer, ., "")",Blake
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Piper, ,, pipe, that, song, again)",Blake
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(;, "", So, I, piped, :, he, wept, to, hear, ., "")",Blake


In [21]:
# define model features and target
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

In [37]:
def model_results(model):
    start_time = time.time()
    score = cross_val_score(model,X,Y, cv=5)
    y_pred = cross_val_predict(model,X,Y,cv=5)
    matrix = confusion_matrix(Y,y_pred)
    print('Cross Validation Scores:\n', score)
    print('\nConfusion Matrix:\n', matrix)
    print("\nCross Validation Range: %0.3f to %0.3f" % (score.min(), score.max()))
    print("\nCross Validation Average: %0.3f" % (score.mean()))
    print("\n--- %s seconds ---" % (time.time() - start_time))

### Logistic Regression

In [38]:
lr = LogisticRegression()
model_results(lr)

Cross Validation Scores:
 [0.77       0.85       0.815      0.7979798  0.75252525]

Confusion Matrix:
 [[367 131]
 [ 71 427]]

Cross Validation Range: 0.753 to 0.850

Cross Validation Average: 0.797

--- 4.08660101890564 seconds ---


In [46]:
#  test parameters
parameters = {'penalty': ['l1', 'l2'],
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
lr = LogisticRegression()
lr_cv = GridSearchCV(lr, parameters)

In [47]:
lr_cv.fit(X, Y)
lr_cv.best_params_

{'C': 0.1, 'penalty': 'l2'}

In [48]:
lr = LogisticRegression(penalty='l2', C=0.1)
model_results(lr)

Cross Validation Scores:
 [0.785      0.845      0.81       0.77777778 0.73737374]

Confusion Matrix:
 [[373 125]
 [ 83 415]]

Cross Validation Range: 0.737 to 0.845

Cross Validation Average: 0.791

--- 4.0556640625 seconds ---


### Random Forest

In [40]:
rfc = ensemble.RandomForestClassifier()
model_results(rfc)

Cross Validation Scores:
 [0.695      0.775      0.685      0.74747475 0.68181818]

Confusion Matrix:
 [[327 171]
 [109 389]]

Cross Validation Range: 0.682 to 0.775

Cross Validation Average: 0.717

--- 5.432806015014648 seconds ---


In [43]:
# test parameters with Grid Search CV
parameters = {'n_estimators': np.arange(5, 50, 5),
    'criterion': ["gini", "entropy"]}
rfc = ensemble.RandomForestClassifier()
rfc_cv = GridSearchCV(rfc, parameters)

In [44]:
rfc_cv.fit(X, Y)
rfc_cv.best_params_

{'criterion': 'entropy', 'n_estimators': 40}

In [45]:
rfc = ensemble.RandomForestClassifier(n_estimators=40, criterion='entropy')
model_results(rfc)

Cross Validation Scores:
 [0.69       0.815      0.755      0.74242424 0.7020202 ]

Confusion Matrix:
 [[325 173]
 [ 91 407]]

Cross Validation Range: 0.690 to 0.815

Cross Validation Average: 0.741

--- 8.242355823516846 seconds ---


### KNN

In [54]:
knn = neighbors.KNeighborsClassifier()

model_results(knn)

Cross Validation Scores:
 [0.65       0.565      0.645      0.65656566 0.58080808]

Confusion Matrix:
 [[163 335]
 [ 44 454]]

Cross Validation Range: 0.565 to 0.657

Cross Validation Average: 0.619

--- 10.338939905166626 seconds ---


In [57]:
# test parameters with Grid Search CV
parameters = {'n_neighbors': np.arange(5, 50, 5),
    'weights': ["uniform", "distance"],
    'algorithm': ["ball_tree", "kd_tree", "brute", "auto"]}
knn = neighbors.KNeighborsClassifier()
knn_cv = GridSearchCV(knn, parameters)

In [58]:
knn_cv.fit(X, Y)
knn_cv.best_params_

{'algorithm': 'ball_tree', 'n_neighbors': 5, 'weights': 'uniform'}

In [59]:
knn = neighbors.KNeighborsClassifier(algorithm='ball_tree', n_neighbors=5, weights='uniform')

model_results(knn)

Cross Validation Scores:
 [0.63       0.56       0.61       0.68686869 0.57575758]

Confusion Matrix:
 [[206 292]
 [ 94 404]]

Cross Validation Range: 0.560 to 0.687

Cross Validation Average: 0.613

--- 9.783508062362671 seconds ---


### Gradient Boost

In [60]:
gb = ensemble.GradientBoostingClassifier()
model_results(gb)

Cross Validation Scores:
 [0.72       0.795      0.765      0.72727273 0.64646465]

Confusion Matrix:
 [[313 185]
 [ 79 419]]

Cross Validation Range: 0.646 to 0.795

Cross Validation Average: 0.731

--- 92.31804203987122 seconds ---


## Latent Semantic Analysis

### Data Processing

In [63]:
poem = gutenberg.paras('blake-poems.txt')
leave = gutenberg.paras('whitman-leaves.txt')

In [74]:
poem_paras=[]
leave_paras=[]
for paragraph in poem:
    para=paragraph[0]
    #removing the double-dash from all words
    para=[re.sub(r'--','',word) for word in para]
    #Forming each paragraph into a string and adding it to the list of strings.
    poem_paras.append(' '.join(para))
for paragraph in leave:
    para=paragraph[0]
    #removing the double-dash from all words
    para=[re.sub(r'--','',word) for word in para]
    #Forming each paragraph into a string and adding it to the list of strings.
    leave_paras.append(' '.join(para))

print(poem_paras[0:4])
print(leave_paras[0:4])

['[ Poems by William Blake 1789 ]', 'SONGS OF INNOCENCE AND OF EXPERIENCE and THE BOOK of THEL', 'SONGS OF INNOCENCE', 'INTRODUCTION']
['[ Leaves of Grass by Walt Whitman 1855 ]', "Come , said my soul , Such verses for my Body let us write , ( for we are one ,) That should I after return , Or , long , long hence , in other spheres , There to some group of mates the chants resuming , ( Tallying Earth ' s soil , trees , winds , tumultuous waves ,) Ever with pleas ' d smile I may keep on , Ever and ever yet the verses owning  as , first , I here and now Signing for Soul and Body , set to them my name ,", 'Walt Whitman', '[ BOOK I . INSCRIPTIONS ]']


In [75]:
poems_para = [[para, "Blake"] for para in poem_para]
leaves_para = [[para, "Whitman"] for para in leave_para]

print(len(poems_para))
print(len(leaves_para))

284
2478


In [76]:
leaves_para = leaves_para[0:len(poems_para)]

# Combine the sentences from the two novels into one data frame.
paras = pd.DataFrame(poems_para + leaves_para)
paras.head()

,0,1
0,"[[[, Poems, by, William, Blake, 1789, ]]]",Blake
1,"[[SONGS, OF, INNOCENCE, AND, OF, EXPERIENCE, a...",Blake
2,"[[SONGS, OF, INNOCENCE]]",Blake
3,[[INTRODUCTION]],Blake
4,"[[Piping, down, the, valleys, wild, ,, Piping,...",Blake


In [77]:
vectorizer = TfidfVectorizer(max_df=0.5, # drop words that occur in more than half the paragraphs
                             min_df=2, # only use words that appear at least twice
                             stop_words='english', 
                             lowercase=True, #convert everything to lower case (since Alice in Wonderland has the HABIT of CAPITALIZING WORDS for EMPHASIS)
                             use_idf=True,#we definitely want to use inverse document frequencies in our weighting
                             norm=u'l2', #Applies a correction factor so that longer paragraphs and shorter paragraphs get treated equally
                             smooth_idf=True #Adds 1 to all document frequencies, as if an extra document existed that used every word once.  Prevents divide-by-zero errors
                            )

poetry_tfidf=vectorizer.fit_transform(paras[0])

AttributeError: 'list' object has no attribute 'lower'